##**Introduction to folium - visualizing data in maps**

**Documentation**: https://pypi.org/project/folium/

**Installation**:

`$ pip install folium`

*OR*

`$ conda install -c conda-forge folium`



<br>

*Open the notebook in **GoogleColab** to check the outputs!*.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1WhfpouOQ-GFmfIkdlrvo83oxWtWjucRZ?usp=sharing)


In [ ]:
# installing folium
#!pip install folium

**Basic features**

In [ ]:
import folium as fl

1. <u>Specific location using latitude and longitude</u>

In [ ]:
# visualizing a specific location using latitude and longitude (Winnipeg in this case)
map = fl.Map(width=500,height=500, location=[49.8951, -97.1384], zoom_start=12)
# tiles are set to OpenStreetMap by default but several are built-in
# maps are generated in large size, use width and height to customize it
map

2. <u>Adding markers</u>

In [ ]:
# adding markers
tooltip = "Details"

# basic marker
fl.Marker([49.8890, -97.1445], popup="<i>RBC Convention Centre</i>", tooltip=tooltip).add_to(map)
# marker with icon
fl.Marker([49.9098, -97.2365], popup="<b>Winnipeg James Armstrong Richardson International Airport</b>", icon=fl.Icon(color="red"), tooltip=tooltip).add_to(map)
map

3. <u>Adding circles</u>

In [ ]:
# adding circles
fl.Circle(
    radius=400,
    location=[49.8280, -97.1146],
    popup="St. Vital Park",
    color="red",
    fill=False,
    #fill_color="#3186cc",
).add_to(map)

map

4. <u>Adding layers</u>

In [ ]:
# adding layers

# for this, we need the boundaries information, aka polygons.
# this files can be found on the Internet in a variety of formats, in this case I found this geojson file 
# in https://data.winnipeg.ca/City-Planning/Neighbourhood/fen6-iygi

# path to the file containing boundaries info
wpg_neighborhoods = f"sample_data/winnipeg_neighbourhood.geojson"

# creating a GeoJSON layer
fl.GeoJson(wpg_neighborhoods, name="geojson").add_to(map)
fl.LayerControl().add_to(map)

map

**Visualizing Data from Pandas Dataframe in Maps**

In [ ]:
# loading sample dataset
import pandas as pd

ga_data = pd.read_csv('sample_data/georgia_real_estate.csv')
ga_data.head()

In [ ]:
#!pip install geopandas

In [ ]:
# loading the geojson file
# source: https://maps.princeton.edu/catalog/tufts-gacounties10

# !pip install geopandas
import geopandas as gpd

# path to the file
ga_counties = f"sample_data/gacounties10-geojson.json"

# let's check the content of the geo file:
gageo_df = gpd.read_file(ga_counties)
gageo_df.head()

In [ ]:
# plotting these boundaries
map_ga = fl.Map(width=600,height=600, location=[32.1656, -82.9001], zoom_start=7, tiles="cartodbpositron") # georgia lat and lon in location

ga_counties = f"sample_data/gacounties10-geojson.json"

# creating a GeoJSON layer
fl.GeoJson(ga_counties, name="geojson").add_to(map_ga)
fl.LayerControl().add_to(map_ga)
map_ga

**Plotting the location of specific properties using markers**

In [ ]:
# plotting properties with price > 30,000
def add_marker_high_price(m, row):
  if(row.price > 30000):
    pp = "County: %s Address: %s Price: %g" %(row.county, row.streetAddress, row.price)
    fl.Marker([row.latitude, row.longitude], popup=pp, icon=fl.Icon(color="red", icon='home', prefix='fa'), tooltip="Details").add_to(m)

# available icons for markers can be found here: https://fontawesome.com/v4/icons/

ga_data.apply(lambda row: add_marker_high_price(map_ga, row), axis=1)
map_ga

**Folium Choropleth**

In [ ]:
# plotting according to polygons
# in this case, let's analyze the price range per county

map_ga_1 = fl.Map(width=600,height=600, location=[32.1656, -82.9001], zoom_start=7, tiles="cartodbpositron")
bins = list(ga_data["price"].quantile([0, 0.25, 0.5, 0.75, 1]))

fl.Choropleth(
    geo_data=ga_counties, # file with geo data
    name="choropleth",
    data=ga_data, # pandas dataframe with the data to be plotted
    columns=["county", "price"], # first column is the key, second is the value to be plotted
    key_on="feature.properties.namelsad10", # field from the geo data file to be matched with the key above
    fill_color="BuPu",
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name="Price",
    bins=bins, # default 6
    highlight=True,
).add_to(map_ga_1)

fl.LayerControl().add_to(map_ga_1)

map_ga_1

**Folium Heatmap**

In [ ]:
from folium.plugins import HeatMap

# initializing map
heat_map = fl.Map(width=500,height=500, location=[32.1656, -82.9001], zoom_start=7, tiles="Stamen Toner")

# folium hetmaps require a list of points of the form [lat, lng] or [lat, lng, weight])
heat_data = [[row['latitude'],row['longitude']] for index, row in ga_data.iterrows()]
# adding heatmap layer to the map
HeatMap(heat_data).add_to(heat_map)
heat_map

In [ ]:
# saving maps: results in an HTML file that allows interactive visualization
map_ga_1.save("map_georgia.html")